# Web scraping. Capítulo 0
## Consumindo API REST con requests

### Instalar as dependencias

Imos empregar requests.

In [1]:
import requests

enderezo = "http://api.open-notify.org/astros.json"
resposta = requests.get(enderezo)


print (resposta)

<Response [200]>


In [2]:
# print(resposta.content)
# print(resposta.text)
resposta.json()

{'message': 'success',
 'people': [{'name': 'Jasmin Moghbeli', 'craft': 'ISS'},
  {'name': 'Andreas Mogensen', 'craft': 'ISS'},
  {'name': 'Satoshi Furukawa', 'craft': 'ISS'},
  {'name': 'Konstantin Borisov', 'craft': 'ISS'},
  {'name': 'Oleg Kononenko', 'craft': 'ISS'},
  {'name': 'Nikolai Chub', 'craft': 'ISS'},
  {'name': "Loral O'Hara", 'craft': 'ISS'}],
 'number': 7}

In [3]:
import json
variables=json.loads(json.dumps(resposta.json()))


for i in resposta.json():
    print(i)
    print (variables[i])

message
success
people
[{'name': 'Jasmin Moghbeli', 'craft': 'ISS'}, {'name': 'Andreas Mogensen', 'craft': 'ISS'}, {'name': 'Satoshi Furukawa', 'craft': 'ISS'}, {'name': 'Konstantin Borisov', 'craft': 'ISS'}, {'name': 'Oleg Kononenko', 'craft': 'ISS'}, {'name': 'Nikolai Chub', 'craft': 'ISS'}, {'name': "Loral O'Hara", 'craft': 'ISS'}]
number
7


In [4]:
if (variables['message'] == 'success'):
    print('Éxito rotundo obtendo datos!')
    print(f"Teño {variables['number']} pilotos!")
    for piloto in variables['people']:
        print(piloto['name'], "tripulación do:", piloto['craft'])

Éxito rotundo obtendo datos!
Teño 7 pilotos!
Jasmin Moghbeli tripulación do: ISS
Andreas Mogensen tripulación do: ISS
Satoshi Furukawa tripulación do: ISS
Konstantin Borisov tripulación do: ISS
Oleg Kononenko tripulación do: ISS
Nikolai Chub tripulación do: ISS
Loral O'Hara tripulación do: ISS


Baseado en: <https://www.nylas.com/blog/use-python-requests-module-rest-apis/>

**Exercicio**: Do servizo: <https://rickandmortyapi.com/api/episode/> podes obter os episodios e personaxes de cada un deles.

Queremos saber aqueles personaxes que participan en **tódolos** episodios.

In [5]:
enderezo = 'https://rickandmortyapi.com/api/episode/'
resposta = requests.get(enderezo)
print(resposta)

<Response [200]>


In [7]:
resposta.json()

{'info': {'count': 51,
  'pages': 3,
  'next': 'https://rickandmortyapi.com/api/episode/?page=2',
  'prev': None},
 'results': [{'id': 1,
   'name': 'Pilot',
   'air_date': 'December 2, 2013',
   'episode': 'S01E01',
   'characters': ['https://rickandmortyapi.com/api/character/1',
    'https://rickandmortyapi.com/api/character/2',
    'https://rickandmortyapi.com/api/character/35',
    'https://rickandmortyapi.com/api/character/38',
    'https://rickandmortyapi.com/api/character/62',
    'https://rickandmortyapi.com/api/character/92',
    'https://rickandmortyapi.com/api/character/127',
    'https://rickandmortyapi.com/api/character/144',
    'https://rickandmortyapi.com/api/character/158',
    'https://rickandmortyapi.com/api/character/175',
    'https://rickandmortyapi.com/api/character/179',
    'https://rickandmortyapi.com/api/character/181',
    'https://rickandmortyapi.com/api/character/239',
    'https://rickandmortyapi.com/api/character/249',
    'https://rickandmortyapi.com/ap

In [8]:
variables=json.loads(json.dumps(resposta.json()))


for i in resposta.json():
    print(i)
    print (variables[i])

info
{'count': 51, 'pages': 3, 'next': 'https://rickandmortyapi.com/api/episode/?page=2', 'prev': None}
results
[{'id': 1, 'name': 'Pilot', 'air_date': 'December 2, 2013', 'episode': 'S01E01', 'characters': ['https://rickandmortyapi.com/api/character/1', 'https://rickandmortyapi.com/api/character/2', 'https://rickandmortyapi.com/api/character/35', 'https://rickandmortyapi.com/api/character/38', 'https://rickandmortyapi.com/api/character/62', 'https://rickandmortyapi.com/api/character/92', 'https://rickandmortyapi.com/api/character/127', 'https://rickandmortyapi.com/api/character/144', 'https://rickandmortyapi.com/api/character/158', 'https://rickandmortyapi.com/api/character/175', 'https://rickandmortyapi.com/api/character/179', 'https://rickandmortyapi.com/api/character/181', 'https://rickandmortyapi.com/api/character/239', 'https://rickandmortyapi.com/api/character/249', 'https://rickandmortyapi.com/api/character/271', 'https://rickandmortyapi.com/api/character/338', 'https://rickand

In [20]:





# Paxinación en info, na parte de arriba de episodios
def procesar_paxina(aux_url):
    aux = requests.get(aux_url)
    variable=json.loads(json.dumps(aux.json()))
    # for i in resposta.json():
        # print(i)
        # print (variables[i])
    for i in aux.json():
      print(variable[i])

def procesar_lista_paxinas(aux_url):
    # Hai seguinte? info -> next -> ata null
    procesar_paxina(aaa)

procesar_paxina('https://rickandmortyapi.com/api/episode/?page=2')
# chapter_URL = '...'
# episode_URL = '...'
# 
# for episodio in variables['results']:
    # print(f'Del episodio: {variables['name']} se encuentran los siguientes personajes:\n')
    # num_personajes = len(variables['characters'])
    # for personaje in variables['characters']:
        # resposta2 = requests.get(personaje)
        # variables2=json.loads(json.dumps(resposta1.json()))
        # for i in resposta1.json():
            # print(i)
            # print (variables[i])

{'count': 51, 'pages': 3, 'next': 'https://rickandmortyapi.com/api/episode/?page=3', 'prev': 'https://rickandmortyapi.com/api/episode/?page=1'}
[{'id': 21, 'name': 'The Wedding Squanchers', 'air_date': 'October 4, 2015', 'episode': 'S02E10', 'characters': ['https://rickandmortyapi.com/api/character/1', 'https://rickandmortyapi.com/api/character/2', 'https://rickandmortyapi.com/api/character/3', 'https://rickandmortyapi.com/api/character/4', 'https://rickandmortyapi.com/api/character/5', 'https://rickandmortyapi.com/api/character/23', 'https://rickandmortyapi.com/api/character/47', 'https://rickandmortyapi.com/api/character/55', 'https://rickandmortyapi.com/api/character/75', 'https://rickandmortyapi.com/api/character/102', 'https://rickandmortyapi.com/api/character/130', 'https://rickandmortyapi.com/api/character/131', 'https://rickandmortyapi.com/api/character/133', 'https://rickandmortyapi.com/api/character/194', 'https://rickandmortyapi.com/api/character/199', 'https://rickandmortya

**Máis información:**
   - <https://www.chakray.com/es/que-diferencias-hay-entre-rest-y-soap/>
   - <https://blog.hubspot.es/website/que-es-api-rest>
   - <https://www.redhat.com/es/topics/integration/whats-the-difference-between-soap-rest>
   - <https://es.martech.zone/what-is-an-api/>